In [1]:
__author__ = 'Roman'
# -*- coding: UTF-8 -*-

import datetime
from urllib.request import urlopen
import time
import json, sys, codecs
import pandas as pd
import numpy as np

%run "../../conn_settings.ipynb"

In [2]:
def get_friends(uid = default_id,  count = 9999):
    reader = codecs.getreader("utf-8")
    raw_data = urlopen(API + 'friends.get?user_id=%s'
                             '&order=name'
                             '&count=%s'
                             '&v=5.59'
                             '&fields=sex,bdate,city,country,has_mobile,contacts'
                             '&access_token=%s' % (uid, count, token)).read().decode()
    
    data = json.loads(raw_data)
    time.sleep(sleep)
    
    return data['response']['items'] 

In [3]:
def get_full_friends(uid = default_id,  count = 9999):
    reader = codecs.getreader("utf-8")
    raw_data = urlopen(API + 'friends.get?user_id=%s'
                             '&order=name'
                             '&count=%s'
                             '&v=5.59'
                             '&fields=sex,bdate,city,country,has_mobile,contacts,domain,followers_count,friend_status,games,'
                             'education,relation,last_seen,status,universities,career,exports,common_count,connections,counters,'
                             'has_photo,home_town,interests,is_friend,is_hidden_from_feed,maiden_name,military,movies,music,'
                             'nickname,occupation,online,personal,quotes,relatives,relation,schools,screen_name,site,tv,verified,wall_comments'
                             '&access_token=%s' % (uid, count, token)).read().decode()
    
    data = json.loads(raw_data)
    time.sleep(sleep)
    
    return data['response']['items'] 

In [8]:
def main():
    return pd.DataFrame.from_dict(get_friends(default_id))
    
df = main()

df.ix[df.city.isnull(), 'city'] = df.ix[df.city.isnull(), 'city'].apply(lambda x: 
                                                                            {'id': -1, 'title': '-'})
df[['city_id','city_name']] = df['city'].apply(pd.Series)
df.drop(['city','city_id'], axis=1, inplace=True)
    
df.ix[df.country.isnull(), 'country'] = df.ix[df.country.isnull(), 'country'].apply(lambda x: 
                                                                                        {'id': -1, 'title': '-'})
df[['country_id','country_name']] = df['country'].apply(pd.Series)
df.drop(['country','country_id'], axis=1, inplace=True)

df.has_mobile.fillna(-1, inplace=True)
df.has_mobile = df.has_mobile.astype(int)

df.home_phone = df.home_phone.str.strip()
df.home_phone.fillna('-', inplace=True)
df.home_phone.replace('','-', inplace=True)

df.deactivated.fillna('OK', inplace=True)

df = df[['id', 'first_name', 'last_name', 'bdate', 'sex', 'city_name', 'country_name', 'has_mobile', 'home_phone', 'deactivated']]
df.to_csv('../../friends_base.csv',index=False, sep=';',encoding='utf-8')

# df.head()